In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from mlxtend.preprocessing import minmax_scaling
import pyttsx3
from pycaret.classification import setup
from pycaret.classification import compare_models

In [11]:
df = pd.read_csv("train.csv")
df.drop(["ID"],axis=1,inplace=True)


In [12]:
from pandas_profiling import ProfileReport

In [13]:
#profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)

In [14]:
#profile

In [15]:
for i in ['Gender', 'Region_Code', 'Occupation', 'Channel_Code','Is_Active']:
    le = LabelEncoder()
    le.fit(df[i])
    df[i]= pd.Series(le.transform(df[i]))

In [16]:
X=df.drop(['Is_Lead'],axis=1)
y=df["Is_Lead"]

In [17]:
X.columns

Index(['Gender', 'Age', 'Region_Code', 'Occupation', 'Channel_Code', 'Vintage',
       'Credit_Product', 'Avg_Account_Balance', 'Is_Active'],
      dtype='object')

In [18]:
temp=list()
for i in range(len(df['Credit_Product'])):
    if df['Credit_Product'][i] == 'Yes' :
        temp.append(0)
    elif df['Credit_Product'][i] == 'No':
        temp.append(0)
    else:
        temp.append(np.nan)
df['Credit_Product'] = temp
X['Credit_Product']=df['Credit_Product']

In [19]:
df['Credit_Product'].value_counts()

0.0    216400
Name: Credit_Product, dtype: int64

In [20]:
X['Credit_Product']=df['Credit_Product']

In [21]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

In [22]:
imputer = KNNImputer(n_neighbors=2)
#X_trans = imputer.fit_transform(X)

In [23]:
X_trandf = pd.DataFrame(X_trans)
X_trandf.columns = X.columns

NameError: name 'X_trans' is not defined

In [24]:
X_trandf.head()


NameError: name 'X_trandf' is not defined

In [25]:
imp_mean = IterativeImputer(random_state=0)
X_iter= imp_mean.fit_transform(X)
X_iter = pd.DataFrame(X_iter)
X_iter.columns = X.columns
X_iter.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
0,0.0,73.0,18.0,1.0,2.0,43.0,0.0,1045696.0,0.0
1,0.0,30.0,27.0,2.0,0.0,32.0,0.0,581988.0,0.0
2,0.0,56.0,18.0,3.0,2.0,26.0,0.0,1484315.0,1.0
3,1.0,34.0,20.0,2.0,0.0,19.0,0.0,470454.0,0.0
4,0.0,30.0,32.0,2.0,0.0,33.0,0.0,886787.0,0.0


In [26]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [27]:
over = SMOTE()
under = RandomUnderSampler()
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
X, y = pipeline.fit_resample(X_iter, y)

In [28]:
y.value_counts()

1    187437
0    187437
Name: Is_Lead, dtype: int64

In [29]:
X.shape

(374874, 9)

In [30]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import SVC
from numpy import mean

In [33]:
tr = pd.DataFrame(X)

In [34]:
tr['Is_Lead']=y

In [35]:
tr.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,0.0,73.0,18.0,1.0,2.0,43.0,0.0,1045696.0,0.0,0
1,0.0,30.0,27.0,2.0,0.0,32.0,0.0,581988.0,0.0,0
2,0.0,56.0,18.0,3.0,2.0,26.0,0.0,1484315.0,1.0,0
3,1.0,34.0,20.0,2.0,0.0,19.0,0.0,470454.0,0.0,0
4,0.0,30.0,32.0,2.0,0.0,33.0,0.0,886787.0,0.0,0


In [37]:
tr.to_csv("smotetrain.csv",index=False)

In [45]:
df = pd.read_csv("test.csv")
df.drop(["ID"],axis=1,inplace=True)

In [46]:
df.shape

(105312, 9)

In [41]:
for i in ['Gender', 'Region_Code', 'Occupation', 'Channel_Code','Is_Active']:
    le = LabelEncoder()
    le.fit(df[i])
    df[i]= pd.Series(le.transform(df[i]))

In [42]:
X=df
temp=list()
for i in range(len(df['Credit_Product'])):
    if df['Credit_Product'][i] == 'Yes' :
        temp.append(0)
    elif df['Credit_Product'][i] == 'No':
        temp.append(0)
    else:
        temp.append(np.nan)
df['Credit_Product'] = temp
X['Credit_Product']=df['Credit_Product']
imp_mean = IterativeImputer(random_state=0)
X_iter= imp_mean.fit_transform(X)
X_iter = pd.DataFrame(X_iter)
X_iter.columns = X.columns
X_iter.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
0,1.0,29.0,4.0,1.0,0.0,25.0,0.0,742366.0,0.0
1,1.0,43.0,18.0,1.0,1.0,49.0,0.0,925537.0,0.0
2,1.0,31.0,20.0,2.0,0.0,14.0,0.0,215949.0,0.0
3,1.0,29.0,22.0,1.0,0.0,33.0,0.0,868070.0,0.0
4,0.0,29.0,20.0,1.0,0.0,19.0,0.0,657087.0,0.0


In [43]:
df.shape,X_iter.shape

((105312, 9), (105312, 9))

In [44]:
X_iter.to_csv("xtest.csv",index=False)

In [ ]:
model = SVC(gamma='scale', class_weight='balanced')
# wrap the model)
calibrated = CalibratedClassifierCV(model, method='isotonic', cv=3)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(calibrated, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.3f' % mean(scores))
import pyttsx3
engine = pyttsx3.init()
newVoiceRate = 100
engine.setProperty('rate',newVoiceRate)
engine.say(10*"Lok Sundar ")
engine.runAndWait()

In [31]:
import pyttsx3
engine = pyttsx3.init()
newVoiceRate = 95
engine.setProperty('rate',newVoiceRate)
engine.say(1*"Lok Sundar ")
engine.runAndWait()

In [ ]:
print("Change power settings")